In [1]:
import  warnings
from    selenium import webdriver
from    selenium.webdriver.common.keys import Keys 
from    selenium.webdriver.common.action_chains import ActionChains
from    selenium.common.exceptions import NoSuchElementException,StaleElementReferenceException
from    bs4      import BeautifulSoup
import  time
import  pyperclip
import  requests
import  datetime
import  pymssql
import  pandas as pd
import  chromedriver_autoinstaller
import  subprocess
from    selenium import webdriver
from    selenium.webdriver.chrome.options import Options
from    selenium.webdriver.common.alert import Alert
import  chromedriver_autoinstaller
import  subprocess
import  shutil 
import  xlrd
import  openpyxl

warnings.filterwarnings('ignore')

f =  open('C:/Users/Gravity/Desktop/python/raw/itemmania_ragnarok_zero.csv', 'w', encoding='utf-8-sig') #최종 csv파일 입력
input_URL = open('C:/Users/Gravity/Desktop/python/raw/input_URL_2.txt', 'w', encoding='utf-8') 
f.write("game,server,is_trade_finished,zeny_sell_title,zeny,cash,regdate,"+'\n')

try:

    shutil.rmtree(r"c:\chrometemp")  #쿠키 / 캐쉬파일 삭제

except FileNotFoundError:

    pass

subprocess.Popen(r'C:\Program Files\Google\Chrome\Application\chrome.exe --remote-debugging-port=9222 --user-data-dir="C:\chrometemp"') # 디버거 크롬 구동

option = Options()

option.add_experimental_option("debuggerAddress", "127.0.0.1:9222")

chrome_ver = chromedriver_autoinstaller.get_chrome_version().split('.')[0]

try:

    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

except:

    chromedriver_autoinstaller.install(True)

    driver = webdriver.Chrome(f'./{chrome_ver}/chromedriver.exe', options=option)

driver.implicitly_wait(10)

url = 'https://trade.itemmania.com/default.html'

driver.get(url)
time.sleep(3)
driver.implicitly_wait(10)

action = ActionChains(driver) #액션지정

#id, pw 입력할 곳 찾기

tag_id = driver.find_element_by_name('user_id')
tag_pw = driver.find_element_by_name('user_password')
driver.implicitly_wait(10)

#id 입력

tag_id.click()
pyperclip.copy('id')
tag_id.send_keys(Keys.CONTROL, 'v')
time.sleep(2)
driver.implicitly_wait(10)

#pw 입력

tag_pw.click()
pyperclip.copy('pw')
tag_pw.send_keys(Keys.CONTROL, 'v')
time.sleep(2)
driver.implicitly_wait(10)

#로그인 버튼 클릭

login_btn = driver.find_element_by_class_name('btn')
login_btn.click()
time.sleep(4)
driver.implicitly_wait(10)

#로그인 완료

#검색창 입력

tag_search_box = driver.find_element_by_name('searchGameServer') #검색창 태그 찾기
tag_search_box.click()  #검색창 클릭
pyperclip.copy('라그나로크제로') 
time.sleep(3)
tag_search_box.send_keys(Keys.CONTROL, 'v') #라그나로크 입력
time.sleep(3)
tag_search_box.send_keys(Keys.ENTER)
time.sleep(3)
tag_search_box.send_keys(Keys.ENTER) #엔터 두번 치면 "서버전체" 로 이동함
time.sleep(3)
driver.implicitly_wait(10)

#팝니다_라그나로크_서버전체 내 "게임머니 클릭"

driver.find_element_by_xpath('//*[@id="g_CONTENT"]/div[4]/div[1]').click()   #팝니다_라그나로크_서버전체 내 "게임머니 클릭"
time.sleep(1)
driver.implicitly_wait(10)

#50개더보기 클릭

while True :
    try :
        load_more = driver.find_element_by_css_selector('div.load_more')  #더보기 클릭
        load_more.click()
        time.sleep(2)
    except :
        break
    
time.sleep(5) #더보기 클릭 이후 "마지막 물품입니다" 팝업창 자동 사라짐

html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')
results = soup.find_all("li", {"class" : "link_block"}) #거래종료된 판매글의 거래ID만 긁어오자
for ar in results :
    trade_id = ar.find("div", {"class" : "view_detail"}) #(거래 ID를 가진 속성 class)
    trade_id2 = trade_id.attrs["trade-id"] #trade-ID 긁자
    url = 'http://trade.itemmania.com/sell/view.html?id='+str(trade_id2) #거래판매종료된 URL 생성 
    input_URL.write(str(url) + '\n') #메모장 입력

input_URL.close() #메모장 닫기

#아무거나 상품 클릭
time.sleep(1)

driver.find_element_by_xpath('//*[@id="g_CONTENT"]/div[11]/ul[2]/li[1]/div[2]/a/div').click()
time.sleep(1)
driver.implicitly_wait(10)

file = open('C:/Users/Gravity/Desktop/python/raw/input_URL_2.txt','r',encoding='utf-8')  #URL 메모장 다시 열기 
urls = file.readlines() #메모장 URL 읽기

for ar in urls : 
    trade_url = ar.replace("\n", "")
    driver.get(trade_url)
    time.sleep(2)
    driver.implicitly_wait(10)
    game_server = driver.find_element_by_xpath('//*[@id="frmSell"]/table/tbody/tr[1]/td').text  #게임/서버
    game_server_2 = game_server.replace("> 게임머니", "") #클린징
    game_server_3 = game_server_2.strip() #클린징
    game_server_4 = game_server_3.replace(" ", "") #클린징
    game_server_5 = game_server_4.replace(">", ",") #클린징(완료)
    trade_results = "거래완료" #거래완료여부
    zeny_selling_title = driver.find_element_by_xpath('//*[@id="frmSell"]/table/tbody/tr[2]/td').text #제니판매글제목
    zeny_selling_title_2 = zeny_selling_title.replace(",", "")
    zeny = driver.find_element_by_xpath('//*[@id="frmSell"]/table/tbody/tr[4]/td[1]').text  #판매제니수량
    zeny_2 = zeny.replace(" 제니", "")  #클린징
    zeny_3 = zeny_2.replace(",", "") #클린징
    cash = driver.find_element_by_xpath('//*[@id="frmSell"]/table/tbody/tr[4]/td[2]').text #거래가격
    cash2 = cash.replace("원", "")
    cash3 = cash2.replace(",", "")
    date = driver.find_element_by_xpath('//*[@id="frmSell"]/table/tbody/tr[3]/td[2]').text #등록일
    date_2 = date[:10]
    f.write(game_server_5 +','+ trade_results +','+ zeny_selling_title_2 +','+ zeny_3 +','+ cash3 +','+ date_2+'\n') #완료(csv파일에 넣기)
    time.sleep(1)
    driver.implicitly_wait(10)

file.close() #URL 메모장 닫기

f.close() #csv파일 닫기

driver.close() #크롬창 닫기
driver.quit() #크롬 드라이버 닫기